In [143]:
# ! pip install beautifulsoup4
# ! pip install requests

Update:
+ Finish scrap the product listing page
+ Know where to find the SKU 
+ Need to scrap available switches and price of each product

In [144]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
# page_no = 1
# url_link = "https://mechanicalkeyboards.com/shop/index.php?pg="+str(page_no)+"&l=product_list&c=1&show=100"

# Shop listing page

In [145]:
def get_soup(url_link):
    return BeautifulSoup(requests.get(url_link).text, 'html.parser')


In [146]:
# soup = get_soup(url_link)
# product_soup = soup.select("div.product-name a")
# image_link = soup.select("div.imagebox li a img")
# full_image_url = "https://mechanicalkeyboards.com/shop/" + image_link[0]['src']

# product_name = product_soup[0].text
# product_url = product_soup[0]['href']

# # print(f"Name:{product_name}\nUrl:{product_url}\nImage Url: {full_image_url}")




In [147]:
product_name_list = []
product_url_list = []
product_img_url_list = []

# i=1
# url_link = "https://mechanicalkeyboards.com/shop/index.php?pg="+str(i)+"&l=product_list&c=1&show=100"
# soup = get_soup(url_link)
# product_soup = soup.select("div.product-name a")
# image_link = soup.select("div.imagebox li a img")

for i in range(1,12):
        url_link = "https://mechanicalkeyboards.com/shop/index.php?pg="+str(i)+"&l=product_list&c=1&show=100"
        soup = get_soup(url_link)
        product_soup = soup.select("div.product-name a")
        image_link = soup.select("div.imagebox li a img")
        for (name,url) in zip(product_soup, image_link):
                product_name = name.text
                product_url = name['href']
                full_image_url = "https://mechanicalkeyboards.com/shop/" + url['src']
                product_name_list.append(product_name)
                product_url_list.append(product_url)
                product_img_url_list.append(full_image_url)



In [164]:
print(product_url)

https://mechanicalkeyboards.com/shop/index.php?l=product_detail&p=10227


# Product Detail Page


In [163]:
product_name_detail = []
product_sku = []

#return product name in tag
def get_product_detail(product_url):
    detail_soup = get_soup(product_url)
    product_detail = detail_soup.select_one('div.product-detail h4')
    return product_detail

#return sku in tag
def get_sku(product_url):
    pd_soup = get_soup(product_url)
    sku_id = pd_soup.select_one('div.product-id span')
    return sku_id

# detail = get_product_detail(product_url_list[0])
# print(detail.text)

# for url in product_url_list:
#     for(name,sku) in zip(get_product_detail(url), get_sku(url)):
#         name = name.text
#         sku = sku.text
        
#         product_name_detail.append(name)
#         product_sku.append(sku)


# print(f"Name:{get_product_name_detail(product_url_list[0])}\nSKU:{get_sku(product_url_list[0])}")


# for url in product_url_list:
#     name = get_product_name_detail(url)
#     sku = get_sku(url)
#     product_name_detail.append(name)
#     product_sku.append(sku)





KeyboardInterrupt: 

In [149]:
product_df = pd.DataFrame({
    "name": product_name_list,
    "product_url": product_url_list,
    "product_img_url": product_img_url_list
})

# sku_df = pd.DataFrame({
#     "name":product_name_detail,
#     "sku": product_sku
# })

print(product_df.shape)
# print(sku_df.shape)

(1024, 3)
